In [25]:
import rasterio
import numpy as np
import pandas as pd
from rasterio.crs import CRS
from glob import glob
from rasterio import plot
import rioxarray as rxr
import matplotlib.pyplot as plt
import geopandas as gpd
pd.options.display.max_rows = 10
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
import scipy.io
import copy
%matplotlib inline

In [26]:
def get_anc_df(month):
    cygnss_data = pd.read_csv('C:/Cygnss/report/cygnss_csv/'+month+'_quality_rough_above_thresh.csv')
    if month == 'april':
        ind = (cygnss_data[cygnss_data['sp_lon'] == -98.58905029296876]).index
        cygnss_data.drop(ind, inplace = True)
        cygnss_data.reset_index(drop = True, inplace = True)
    preff = cygnss_data['preff']
    pow_mean = cygnss_data['pow_mean']
    pow_var = cygnss_data['pow_var']
    pow_skew = cygnss_data['pow_skew']
    pow_kurt = cygnss_data['pow_kurt']
    sp_lat = cygnss_data['sp_lat']
    sp_lon = cygnss_data['sp_lon']
    inc_angle = cygnss_data['sp_inc_angle']
    day_num = cygnss_data['day']
    
    soil_files_list = glob('soils/soils/*.tif')
    ele_files_list = glob('3dep_13/3dep_13/*.tif')
    
    ndvi = rasterio.open('report/ancillary/modis_ndvi/'+month+'/ndvi_'+month+'_reprojected.tif')
    elevation = rasterio.open(ele_files_list[0])
    clay = rasterio.open(soil_files_list[0])
    depres = rasterio.open(soil_files_list[1])
    sand = rasterio.open(soil_files_list[2])
    silt = rasterio.open(soil_files_list[3])
    
    elevation_data = elevation.read(1)
    clay_data = clay.read(1)
    depres_data = depres.read(1)
    sand_data = sand.read(1)
    silt_data = silt.read(1)
    ndvi_data = ndvi.read(1)

    clay_data = np.where(
    clay_data < 0,
    0.,
    clay_data)

    sand_data = np.where(
    sand_data < 0,
    0.,
    sand_data)

    silt_data = np.where(
    silt_data < 0,
    0.,
    silt_data)

    depres_data = np.where(
    depres_data < 0,
    0.,
    depres_data)

    eles = []
    cls = []
    snds = []
    sils = []
    deps = []
    ndvis = []
    for i in range(cygnss_data.shape[0]):
            
        lat = sp_lat[i]
        lon = sp_lon[i]



        ele_ind = elevation.index(lon, lat)
#         if ((month=='april')&(ele_ind[1] == 4035)):
#             ele_ind = elevation.index(lon, lat)
#             print(ele_ind)
#             print(lat, lon)
#             ele_ind = (ele_ind[0], 4033)
        clay_ind = clay.index(lon, lat)
        sand_ind = sand.index(lon, lat)
        silt_ind = silt.index(lon, lat)
        depres_ind = depres.index(lon, lat)
        ndvi_ind = ndvi.index(lon, lat)

        eles.append(elevation_data[ele_ind])
        cls.append(clay_data[clay_ind])
        snds.append(sand_data[sand_ind])
        sils.append(silt_data[silt_ind])
        deps.append(depres_data[depres_ind])
        ndvis.append(ndvi_data[ndvi_ind])
        
    d = {'day_num':day_num, 'lat':sp_lat, 'lon': sp_lon,
         'preff':preff, 'inc_angle':inc_angle, 'elevation':eles,
         'clay':cls, 'sand':snds, 'silt':sils, 'depres':deps, 
         'ndvi':ndvis, 'pow_mean': pow_mean, 'pow_var':pow_var, 'pow_skew':pow_skew, 'pow_kurt':pow_kurt}
    
#     d = {'day_num':day_num, 'lat':sp_lat, 'lon': sp_lon,
#          'preff':preff, 'inc_angle':inc_angle, 'elevation':eles,
#          'clay':cls, 'sand':snds, 'silt':sils, 'depres':deps, 
#          'ndvi':ndvis}
    anc_data = pd.DataFrame(d)
    
    return anc_data

In [27]:
jan = get_anc_df('jan')
feb = get_anc_df('feb')
march = get_anc_df('march')
april = get_anc_df('april')
may = get_anc_df('may')
june = get_anc_df('june')
july = get_anc_df('july')
august = get_anc_df('august')
sept = get_anc_df('sept')
octo = get_anc_df('oct')
nov = get_anc_df('nov')
dec = get_anc_df('dec')


jan_clean = jan.dropna()
feb_clean = feb.dropna()
march_clean = march.dropna()
april_clean = april.dropna()
may_clean = may.dropna()
june_clean = june.dropna()
july_clean = july.dropna()
august_clean = august.dropna()
sept_clean = sept.dropna()
oct_clean = octo.dropna()
nov_clean = nov.dropna()
dec_clean = dec.dropna()


In [30]:
np.round(pearsonr(nov_clean['preff'], nov_clean['elevation']),3)

array([-0.416,  0.   ])

In [37]:
month_to_num_days = {'jan':31,
                     'feb':28,
                     'march':31,
                    'april':30,
                     'may':31,
                     'june':30,
                    'july':31,
                    'august':31,
                    'sept':30,
                    'oct':31,
                    'nov':30,
                    'dec':31}

method_to_txson_type = {'idw':'SWCi',
                    'voronoi':'SWCv'}

method_to_cygnss_type = {'idw':'CYGNSSi',
                        'voronoi':'CYGNSSv'}
# month_to_smap_lower = {
#     'jan':1,
#     'april':91,
#     'july':182
# }

# month_to_smap_upper = {
#     'jan':31,
#     'april':120,
#     'july':212
# }

month_range = {
    'jan':(1,31),
    'feb':(32, 59),
    'march':(60, 90),
    'april':(91,120),
    'may':(121,151),
    'june':(152,181),
    'july':(182,212),
    'august':(213,243),
    'sept':(244,273),
    'oct':(274, 304),
    'nov':(305, 334),
    'dec':(335, 365)
}

In [38]:
# Set the required grid resolution here (3km or 9km)
grid_res = '9km'

snr = 0
preff = 1

In [39]:
def get_txson_smap(month, method, grid_res=grid_res):
    
    txson_path = 'C:/Cygnss/TxSON_v1_3/TxSON_v1_3/TxSON_upscaled/txson_' + month + '_'+method +'_'+ grid_res + '.mat'
    txson = scipy.io.loadmat(txson_path)
    
    txson_type = method_to_txson_type[method]
    TxSON = txson[txson_type][:,:,0]
    
#     cygnss_path = 'cygnss_upscaled_2/cygnss_' + month + '_quality_'+ method + '_'+ grid_res + '.mat'
# #     cygnss_path = 'cygnss_upscaled_2/cygnss_' + month + '_' + method + '_'+ grid_res + '.mat'
#     cygnss = scipy.io.loadmat(cygnss_path)
    
#     cygnss_type = method_to_cygnss_type[method]
#     CYGNSS = cygnss[cygnss_type]
    
    
#     month_lower_day = month_to_smap_lower[month]
#     month_upper_day = month_to_smap_upper[month]
    month_lower_day, month_upper_day = month_range[month]
    SMAP = np.empty((month_to_num_days[month], TxSON.shape[1]))
    SMAP[:] = np.NaN
    smap_data_path = 'C:/Cygnss/smap/'+grid_res+'/smap_sm_9km_2019*.tif'
    smap_2019_files_list = glob(smap_data_path)
    smap_month_list = [x for x in smap_2019_files_list if ((int(x[-7:-4])>=month_lower_day) & (int(x[-7:-4])<=month_upper_day))]
    
    for i in range(len(smap_month_list)):
        smap = rasterio.open(smap_month_list[i])
        smap_1 = smap.read(1).astype('float64')
        smap_2 = smap.read(2).astype('float64')
        smap_avg = np.nanmean(np.array([smap_1, smap_2]), axis = 0)
#         smap_avg = (np.add(smap_1, smap_2))/2
    
        #reorder smap data according to the fishnet order of grids
        smap_avg_reordered = smap_avg[[3, 2, 1, 0]]

        smap_avg_flatten = smap_avg_reordered.flatten()
        SMAP[i,:] = smap_avg_flatten
        
    
    return TxSON, SMAP

In [40]:
TxSON_jan, SMAP_jan = get_txson_smap('jan', 'voronoi')
TxSON_feb, SMAP_feb = get_txson_smap('feb', 'voronoi')
TxSON_march, SMAP_march = get_txson_smap('march', 'voronoi')
TxSON_april, SMAP_april = get_txson_smap('april', 'voronoi')
TxSON_may, SMAP_may = get_txson_smap('may', 'voronoi')
TxSON_june, SMAP_june = get_txson_smap('june', 'voronoi')
TxSON_july, SMAP_july = get_txson_smap('july', 'voronoi')
TxSON_august, SMAP_august = get_txson_smap('august', 'voronoi')
TxSON_sept, SMAP_sept = get_txson_smap('sept', 'voronoi')
TxSON_oct, SMAP_oct = get_txson_smap('oct', 'voronoi')
TxSON_nov, SMAP_nov = get_txson_smap('nov', 'voronoi')
TxSON_dec, SMAP_dec = get_txson_smap('dec', 'voronoi')

C:\Users\mathurs\Anaconda3\envs\geo_env_2\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: Mean of empty slice


In [41]:
fishnet_path = 'C:/Cygnss/grid_corners/fishnet_' + grid_res +'.shp'
fishnet = gpd.read_file(fishnet_path)
grid_corners_path = 'C:/Cygnss/grid_corners/fishnet_' + grid_res +'.csv'
grid_corners = pd.read_csv(grid_corners_path)

In [42]:
def get_merged_fishnet(fishnet, month, TxSON, SMAP):
    
    num_days = month_to_num_days[month]
    txson_fishnet = copy.deepcopy(fishnet)
    smap_fishnet = copy.deepcopy(fishnet)
    month_days = np.arange(num_days)
#     for day in month_days:
#         preff_day = CYGNSS[day,:, preff]
#         preff_day_number = 'preff_day_' + str(day+1)
#         merged_fishnet[preff_day_number] = preff_day 
        
    
    cell_num = 16
    grids_month_mean = np.zeros((cell_num,))
    
#     for i in range(16):
#         grid_mean = np.nanmean(CYGNSS[:,i,preff])
#         grids_month_mean[i] = grid_mean
#     merged_fishnet['preff_month_mean'] = grids_month_mean
    
    for day in month_days:
        lower = (day)*24
        upper = lower+24
        txson_fishnet['txson_day_'+str(day+1)] = np.nanmean(TxSON[lower:upper,:], axis = 0)  
#     merged_fishnet['txson_month_mean'] = np.nanmean(TxSON, axis = 0) 
    
    for day in month_days:
        smap_day = SMAP[day, :]
        smap_day_number = 'smap_day_'+str(day+1)
        smap_fishnet[smap_day_number] = smap_day
        
    grids_month_mean = np.zeros((cell_num,))
    for i in range(SMAP.shape[1]):
        grid_mean = np.nanmean(SMAP[:,i])
        grids_month_mean[i] = grid_mean
#     merged_fishnet['smap_month_mean'] = grids_month_mean
        
    
    return txson_fishnet, smap_fishnet   

In [43]:
jan_txson, jan_smap = get_merged_fishnet(fishnet, 'jan', TxSON_jan, SMAP_jan)
feb_txson, feb_smap = get_merged_fishnet(fishnet, 'feb', TxSON_feb, SMAP_feb)
march_txson, march_smap = get_merged_fishnet(fishnet, 'march', TxSON_march, SMAP_march)
april_txson, april_smap = get_merged_fishnet(fishnet, 'april', TxSON_april, SMAP_april)
may_txson, may_smap = get_merged_fishnet(fishnet, 'may', TxSON_may, SMAP_may)
june_txson, june_smap = get_merged_fishnet(fishnet, 'june', TxSON_june, SMAP_june)
july_txson, july_smap = get_merged_fishnet(fishnet, 'july', TxSON_july, SMAP_july)
august_txson, august_smap = get_merged_fishnet(fishnet, 'august', TxSON_august, SMAP_august)
sept_txson, sept_smap = get_merged_fishnet(fishnet, 'sept', TxSON_sept, SMAP_sept)
oct_txson, oct_smap = get_merged_fishnet(fishnet, 'oct', TxSON_oct, SMAP_oct)
nov_txson, nov_smap = get_merged_fishnet(fishnet, 'nov', TxSON_nov, SMAP_nov)
dec_txson, dec_smap = get_merged_fishnet(fishnet, 'dec', TxSON_dec, SMAP_dec)

In [44]:
sept_txson.iloc[3]

Id                                                              0
geometry        POLYGON ((-98.682573 30.149063, -98.682573 30....
txson_day_1                                              0.116579
txson_day_2                                              0.117536
txson_day_3                                              0.118611
                                      ...                        
txson_day_26                                             0.121218
txson_day_27                                             0.119492
txson_day_28                                             0.118177
txson_day_29                                             0.116951
txson_day_30                                             0.117189
Name: 3, Length: 32, dtype: object

In [45]:
def get_txson_gt(month_clean, month_txson, month):
    txson_gt = []
    for i in range(month_clean.shape[0]):
        day_num = int(month_clean.iloc[i]['day_num'])
        sp_lat = month_clean.iloc[i]['lat']
        sp_lon = month_clean.iloc[i]['lon']
        for j in range(grid_corners.shape[0]):
            grid_lat_min = grid_corners.iloc[j]['LAT_corner_min']
            grid_lat_max = grid_corners.iloc[j]['LAT_corner_max']
            grid_lon_min = grid_corners.iloc[j]['LON_corner_min']
            grid_lon_max = grid_corners.iloc[j]['LON_corner_max']
            if ((sp_lat>grid_lat_min)&(sp_lat < grid_lat_max)&(sp_lon > grid_lon_min)&(sp_lon < grid_lon_max)):
                grid_ind = int(j)
    #             print(grid_ind)
                break
        
        month_lower_day, _ = month_range[month]
        txson_gt.append(month_txson.iloc[grid_ind][day_num-month_lower_day+2])

#         txson_gt.append(month_txson.iloc[grid_ind][day_num-month_to_smap_lower[month]+2])
    return txson_gt

In [46]:
jan_txson_gt = get_txson_gt(jan_clean, jan_txson, 'jan')
jan_clean['txson_gt'] = jan_txson_gt

feb_txson_gt = get_txson_gt(feb_clean, feb_txson, 'feb')
feb_clean['txson_gt'] = feb_txson_gt

march_txson_gt = get_txson_gt(march_clean, march_txson, 'march')
march_clean['txson_gt'] = march_txson_gt

april_txson_gt = get_txson_gt(april_clean, april_txson, 'april')
april_clean['txson_gt'] = april_txson_gt

may_txson_gt = get_txson_gt(may_clean, may_txson, 'may')
may_clean['txson_gt'] = may_txson_gt

june_txson_gt = get_txson_gt(june_clean, june_txson, 'june')
june_clean['txson_gt'] = june_txson_gt

july_txson_gt = get_txson_gt(july_clean, july_txson, 'july')
july_clean['txson_gt'] = july_txson_gt

august_txson_gt = get_txson_gt(august_clean, august_txson, 'august')
august_clean['txson_gt'] = august_txson_gt

sept_txson_gt = get_txson_gt(sept_clean, sept_txson, 'sept')
sept_clean['txson_gt'] = sept_txson_gt

oct_txson_gt = get_txson_gt(oct_clean, oct_txson, 'oct')
oct_clean['txson_gt'] = oct_txson_gt

nov_txson_gt = get_txson_gt(nov_clean, nov_txson, 'nov')
nov_clean['txson_gt'] = nov_txson_gt

dec_txson_gt = get_txson_gt(dec_clean, dec_txson, 'dec')
dec_clean['txson_gt'] = dec_txson_gt

C:\Users\mathurs\Anaconda3\envs\geo_env_2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mathurs\Anaconda3\envs\geo_env_2\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\mathurs\Anaconda3\envs\geo_env_2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [47]:
dataset = pd.concat([jan_clean, feb_clean, march_clean, april_clean, may_clean, june_clean, july_clean, august_clean, sept_clean, oct_clean, nov_clean, dec_clean]).reset_index()
# dataset = pd.concat([jan_clean, april_clean, july_clean]).reset_index()
# dataset = dataset.drop(columns=['elevation', 'day_num', 'lat', 'lon', 'sand', 'silt', 'depres', 'ndvi'])

In [48]:
dataset.to_csv('C:/Cygnss/report/ML/data_all_months_9km.csv', index = False)

In [211]:
data = pd.read_csv('C:/Cygnss/report/ML/data.csv')

In [213]:
data.iloc[:,4:-1]

,preff,inc_angle,elevation,clay,sand,silt,depres,ndvi,pow_mean,pow_var,pow_skew,pow_kurt
0,152.839042,37.919052,500.277008,50.0,22.1,27.9,201.0,4562,-690.990051,6432023.0,-3.397408,9.544185
1,152.661684,37.938034,486.428375,17.5,43.0,39.5,201.0,4416,-690.990051,6432023.0,-3.397408,9.544185
2,152.786237,37.957127,480.731262,10.5,76.0,13.5,201.0,3901,-690.990051,6432023.0,-3.397408,9.544185
3,151.391626,37.976330,483.185364,22.5,39.8,37.7,46.0,5288,-690.990051,6432023.0,-3.397408,9.544185
4,150.778259,37.995640,470.681946,7.5,85.9,6.6,201.0,5203,-690.990051,6432023.0,-3.397408,9.544185
...,...,...,...,...,...,...,...,...,...,...,...,...
971,151.772158,34.456806,503.060455,7.5,85.9,6.6,201.0,4771,-632.182190,5922417.0,-3.588071,10.879371
972,151.977943,34.474243,492.882843,17.5,43.0,39.5,201.0,5950,-632.182190,5922417.0,-3.588071,10.879371
973,151.624494,34.491711,493.881653,7.5,85.9,6.6,201.0,6256,-632.182190,5922417.0,-3.588071,10.879371
974,150.942498,34.509193,490.095276,10.5,76.0,13.5,201.0,6574,-632.182190,5922417.0,-3.588071,10.879371
